In [1]:
import ee
import os, shutil
import rasterio
import numpy as np
from pprint import pprint
from IPython.display import display
from IPython.display import Image
import matplotlib.pyplot as plt
from matplotlib import colors

In [2]:
from channel_classifier.internal.ee_utils import add_labels, compute_error_image, get_landsat_image, get_landsat_composite
from channel_classifier.internal.utils import download_rgb_image, download_tiff, read_image_list

In [3]:
import tempfile, urllib, zipfile
def download_tiff(image, filename, scale):
    url = ee.data.makeDownloadUrl(ee.data.getDownloadId({
        'image': image.serialize(),
        'scale': '{:d}'.format(scale),
        'filePerBand': 'false',
        'name': 'data',
    }))
    local_zip, headers = urllib.urlretrieve(url)
    with zipfile.ZipFile(local_zip) as local_zipfile:
        local_zipfile.extract('data.tif', '')
        shutil.copyfile('data.tif', filename)
    return filename

In [24]:
def fix_image(filename):
    with rasterio.open(filename) as r:
        image = r.read()
        profile = r.profile
        profile.update(nodata=-9999)
        image[np.isnan(image)] = profile['nodata']
        image[np.isinf(image)] = profile['nodata']
        red = image[2]
        green = image[1]
        blue = image[0]
    with rasterio.open(filename, 'w', **profile) as out:
        out.write(red, 1)
        out.write(green, 2)
        out.write(blue, 3)

In [5]:
ee.Initialize()

In [6]:
datasets = {'LE7' : 'LANDSAT/LE07/C01/T1_TOA',
            'LT5' : 'LANDSAT/LT05/C01/T1_TOA',
           }

In [7]:
image_list = read_image_list('imagelist.txt')

In [25]:
thresh = 5
filenames = []
for sat, path, row, date in image_list[0:100]:
    dataset_path = datasets[sat]
    year, month, day = map(int, date.split('-'))
    filename = 'pan/{}_{}{}{}_{}{}.tif'.format(sat, year, month, day, path, row)
    
    try:
        im = get_landsat_image(path, row, year, month, day, dataset_path=dataset_path)
        cc = im.get('CLOUD_COVER').getInfo()
        if cc < thresh:
            download_tiff(im.select(['B1', 'B2', 'B3']), filename, scale=120)
            print('Downloaded ' + filename + ' (cc: {}%)'.format(cc))
            fix_image(filename)
    except:
        print('Image {} failed'.format(filename))

Image pan/LE7_201441_664.tif failed
Image pan/LE7_200642_764.tif failed
Image pan/LE7_200343_664.tif failed
Image pan/LE7_201345_764.tif failed
Image pan/LE7_200247_763.tif failed
Image pan/LE7_201448_763.tif failed
Image pan/LE7_201149_664.tif failed
Image pan/LE7_2003410_763.tif failed
Image pan/LE7_2012411_664.tif failed
Image pan/LE7_2015411_763.tif failed
Image pan/LE7_2015411_764.tif failed
Image pan/LE7_2001413_664.tif failed
Image pan/LE7_2013414_664.tif failed
Image pan/LE7_2011416_763.tif failed
Image pan/LE7_2011416_764.tif failed
Image pan/LE7_2012418_764.tif failed
Image pan/LE7_2015420_664.tif failed
Image pan/LE7_2004421_664.tif failed
Image pan/LE7_2002423_764.tif failed
Image pan/LE7_2014424_763.tif failed
Image pan/LE7_2014424_764.tif failed
Image pan/LE7_2011425_664.tif failed
Image pan/LE7_2003426_764.tif failed
Image pan/LE7_2009426_763.tif failed
Image pan/LE7_2009426_764.tif failed
Image pan/LE7_2012427_664.tif failed
Image pan/LE7_2015427_764.tif failed
Image pa